In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import tqdm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os

res0_names = os.listdir('/content/drive/MyDrive/kodiim senior/results_m/result0')
res1_names = os.listdir('/content/drive/MyDrive/kodiim senior/results_m/result1')
res0_test_names = os.listdir('/content/drive/MyDrive/kodiim senior/results_m/result0_test')
res1_test_names = os.listdir('/content/drive/MyDrive/kodiim senior/results_m/result1_test')

In [ ]:
df1 = pd.DataFrame()
for d in res1_names:
  sub_df1 = pd.read_csv('/content/drive/MyDrive/kodiim senior/results_m/result1/' + d)
  df1 = pd.concat([df1, sub_df1], ignore_index=True)

In [ ]:
df0 = pd.DataFrame()
for d in res0_names:
  sub_df0 = pd.read_csv('/content/drive/MyDrive/kodiim senior/results_m/result0/' + d)
  df0 = pd.concat([df0, sub_df0], ignore_index=True)

In [ ]:
df1_test = pd.DataFrame()
for d in res1_test_names:
  sub_df1 = pd.read_csv('/content/drive/MyDrive/kodiim senior/results_m/result1_test/' + d)
  df1_test = pd.concat([df1_test, sub_df1], ignore_index=True)

In [ ]:
df0_test = pd.DataFrame()
for d in res0_test_names:
  sub_df0 = pd.read_csv('/content/drive/MyDrive/kodiim senior/results_m/result0_test/' + d)
  df0_test = pd.concat([df0_test, sub_df0], ignore_index=True)

In [ ]:
res1_2_names = os.listdir('/content/drive/MyDrive/kodiim senior/results_m/result1_2')

In [ ]:
df1_2 = pd.DataFrame()
for d in res1_2_names:
  sub_df1 = pd.read_csv('/content/drive/MyDrive/kodiim senior/results_m/result1_2/' + d)
  df1_2 = pd.concat([df1_2, sub_df1], ignore_index=True)

In [ ]:
df0.shape[0], df1.shape[0]

(18632, 23000)

In [ ]:
df0.isna().sum()

Path       0
Text    9445
dtype: int64

In [ ]:
df1.isna().sum()

Path      0
Text    125
dtype: int64

In [ ]:
df0 = df0.fillna('')
df1 = df1.fillna('')

In [ ]:
item_meta0 = df0

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

In [ ]:
batch_size = 64
n = len(item_meta0)
content_item_embeddings0 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_meta0['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_item_embeddings0.append(sentence_embeddings)

100%|██████████| 292/292 [00:18<00:00, 15.97it/s]


In [ ]:
item_meta1 = df1

In [ ]:
batch_size = 64
n = len(item_meta1)
content_item_embeddings1 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_meta1['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_item_embeddings1.append(sentence_embeddings)

100%|██████████| 360/360 [00:12<00:00, 29.16it/s]


In [ ]:
len(content_item_embeddings1)

360

In [ ]:
embeddings0 = []
for batch in content_item_embeddings0:
  for elem in batch:
    embeddings0.append(elem)

In [ ]:
len(embeddings0)

18632

In [ ]:
embeddings1 = []
for batch in content_item_embeddings1:
  for elem in batch:
    embeddings1.append(elem)

In [ ]:
len(embeddings1)

23000

In [ ]:
df0['embeddings'] = embeddings0

In [ ]:
df1['embeddings'] = embeddings1

In [ ]:
for i in range(312):
  df0[str(i)] = df0.embeddings.apply(lambda x: x[i]);

<ipython-input-134-40eb6fcfb9f5>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df0[str(i)] = df0.embeddings.apply(lambda x: x[i]);
<ipython-input-134-40eb6fcfb9f5>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df0[str(i)] = df0.embeddings.apply(lambda x: x[i]);
<ipython-input-134-40eb6fcfb9f5>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-f

In [ ]:
for i in range(312):
  df1[str(i)] = df1.embeddings.apply(lambda x: x[i]);

<ipython-input-135-0f34f2892539>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[str(i)] = df1.embeddings.apply(lambda x: x[i]);
<ipython-input-135-0f34f2892539>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[str(i)] = df1.embeddings.apply(lambda x: x[i]);
<ipython-input-135-0f34f2892539>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-f

In [ ]:
df0_emb = df0.drop(columns=['embeddings'])
df1_emb = df1.drop(columns=['embeddings'])

In [ ]:
df0_emb.shape[0], df1_emb.shape[0]

(18632, 23000)

In [ ]:
df0_emb.to_csv('df0_with_embeddings_18632.csv', index=False)
df1_emb.to_csv('df1_with_embeddings_23000.csv', index=False)

In [ ]:
len(set(df0_test.Path) & set(df1_test.Path))

0

In [ ]:
df0_test.shape[0], df1_test.shape[0]

(1980, 1252)

In [ ]:
df0_test.isna().sum(), df1_test.isna().sum()

(Path       0
 Text    1017
 dtype: int64,
 Path     0
 Text    19
 dtype: int64)

In [ ]:
df0_test = df0_test.fillna('')
df1_test = df1_test.fillna('')

In [ ]:
item_test0 = df0_test

In [ ]:
batch_size = 64
n = len(item_test0)
content_test_embeddings0 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_test0['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_test_embeddings0.append(sentence_embeddings)

100%|██████████| 31/31 [00:02<00:00, 13.97it/s]


In [ ]:
item_test1 = df1_test

In [ ]:
batch_size = 64
n = len(item_test1)
content_test_embeddings1 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_test1['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_test_embeddings1.append(sentence_embeddings)

100%|██████████| 20/20 [00:01<00:00, 14.39it/s]


In [ ]:
test_embeddings0 = []
for batch in content_test_embeddings0:
  for elem in batch:
    test_embeddings0.append(elem)

In [ ]:
test_embeddings1 = []
for batch in content_test_embeddings1:
  for elem in batch:
    test_embeddings1.append(elem)

In [ ]:
len(test_embeddings0), len(test_embeddings1)

(1980, 1252)

In [ ]:
df0_test['embeddings'] = test_embeddings0
df1_test['embeddings'] = test_embeddings1

In [ ]:
for i in range(312):
  df0_test[str(i)] = df0_test.embeddings.apply(lambda x: x[i])
  df1_test[str(i)] = df1_test.embeddings.apply(lambda x: x[i])

<ipython-input-164-e98463dd4646>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df0_test[str(i)] = df0_test.embeddings.apply(lambda x: x[i])
<ipython-input-164-e98463dd4646>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1_test[str(i)] = df1_test.embeddings.apply(lambda x: x[i])
<ipython-input-164-e98463dd4646>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

In [ ]:
df0_emb_test = df0_test.drop(columns=['embeddings'])
df1_emb_test = df1_test.drop(columns=['embeddings'])

In [ ]:
df0_emb_test.shape[0], df1_emb_test.shape[0]

(1980, 1252)

In [ ]:
df0_emb_test.to_csv('df0_test_with_embeddings_1980.csv', index=False)
df1_emb_test.to_csv('df1_test_with_embeddings_1252.csv', index=False)

In [ ]:
dir = '/content/drive/MyDrive/kodiim senior/test0_with_embeddings_1980/'

In [ ]:
df0_emb_test.to_csv(f'{dir}df0_test_with_embeddings_1980.csv', index=False)